# SEGMENTING AND CLUSTERING 

In [10]:
import pandas as pd
import numpy as np

In [11]:
#!pip install requests
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests

In [12]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
headers = {'User-Agent': 'Mozilla/5.0'}
page = requests.get(url, headers=headers)

In [124]:
soup = BeautifulSoup(page.text, "html.parser")
table = soup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df=pd.DataFrame(l, columns=["Postcode", "Borough", "Neighbourhood"])
df['Neighbourhood']=df['Neighbourhood'].str.replace('\n','')

In [125]:
df=df[df['Borough']!='Not assigned'].dropna()

In [126]:
df=df.groupby(['Postcode', 'Borough']).agg(lambda col: ', '.join(col))#.agg(list)

# Fill NaN in Neighbourhood-Column with Borough-Name

In [138]:
df.shape

(103, 1)